In [ ]:
import numpy as np
import pandas as pd
import pickle

## Load data

In [ ]:
# load in data
with open('all_metrics.p', 'rb') as f:
    all_metrics = pickle.load(f)

In [ ]:
# convert to rows of data
metrics_distilled = {}
functions = [("mean", np.mean), ("median", np.median), ("std", np.std)]
for run in all_metrics:
    if run == "baseline":
        continue
    metrics_this_run = {}
    for metric in all_metrics[run]["metrics"]:
        for func_name, func in functions:
            if isinstance(all_metrics[run]["metrics"][metric], dict):
                for agent, values in all_metrics[run]["metrics"][metric].items():
                    met_agent_name = "{}_{}_{}".format(metric, agent, func_name)
                    metrics_this_run[met_agent_name] = func(values)
            else:
                met_agent_name = "{}_{}".format(metric, func_name)
                metrics_this_run[met_agent_name] = func(all_metrics[run]["metrics"][metric])
    metrics_distilled[run] = metrics_this_run

# convert to dataframe for visualization
df = pd.DataFrame.from_dict(metrics_distilled).T
print(df.shape)
df

## Turn into a table

In [ ]:
subrow_formatter = '\tworowsubtablecenter{{{}}}{{\tworowsubtablecenter{{{}}}{{{}}}}}'
mean_or_median = "mean"

results_tex = []
for i, (run, metrics) in enumerate(metrics_distilled.items()):
    coordinated = not "un" in run
    n_attackers = run.split("_")[1]
    metrics_as_strings = {}
    metric_names = {met_name.replace("_mean", "").replace("_median", "").replace("_std", "") for met_name in metrics}
    for met_name in metric_names:
        met_name_tex = met_name.replace("_", "\_")
        if "agent" in met_name:
            agent_num = int(met_name.split("_")[1].replace("agent", ""))
            if agent_num > int(n_attackers):
                metrics_as_strings[met_name_tex] = "N/A"
                continue
        metrics_as_strings[met_name_tex] = "{:4.2f} +/- {:4.2f}".format(
            metrics["{}_{}".format(met_name, mean_or_median)],
            metrics["{}_{}".format(met_name, "std")],
        )
    sorted_metrics = dict(sorted(metrics_as_strings.items()))
    tex_row = {
        "Run ID": "{}-{}".format("C" if coordinated else "UC", n_attackers),
        "Coord?": coordinated,
        "\# Adv": n_attackers,
        **sorted_metrics,
    }
    results_tex.append(tex_row)

In [ ]:
df_tex_ca

In [ ]:
# convert to df and drop all nan columns
df_tex = pd.DataFrame(results_tex)
for col in df_tex:
    if (df_tex[col] == df_tex[col][0]).all():
        df_tex.drop(columns=[col], inplace=True)

# separate into two dataframes -- one for CAFN/FP
col_ca = [col for col in df_tex.columns if "CAF" in col]
col_er = [col for col in df_tex.columns if "ERCC" in col]
df_tex_er = df_tex.drop(columns=col_ca)
df_tex_ca = df_tex.drop(columns=col_er)

# convert to tex
pd.set_option('display.max_colwidth',1000)
tex_string_er = df_tex_er.to_latex(index=False, multirow=True, escape=False).replace('\\\\\n', '\\\\ \\midrule\n')
tex_string_ca = df_tex_ca.to_latex(index=False, multirow=True, escape=False).replace('\\\\\n', '\\\\ \\midrule\n')
print("\\begin{subtable}[t]{\\linewidth}\n\\centering")
print(tex_string_er, end="")
print("\\caption{\\panelonesubcaption}")
print("\\end{subtable}")
print("\\begin{subtable}[t]{\\linewidth}\n\\centering")
print(tex_string_ca, end="")
print("\\caption{\\paneltwosubcaption}")
print("\\end{subtable}")